In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("LogisticRegression").getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [8]:
my_data=spark.read.csv("C:/Users/User/Desktop/Data/flights.csv", header=True, inferSchema=True)

In [9]:
my_data.show(5)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 5 rows



In [10]:
my_data.printSchema()

root
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)



In [13]:
flightschema=StructType([
   StructField ("DayofMonth", IntegerType(),False),
    StructField("DayofWeek", IntegerType(),False),
    StructField("Carrier", StringType(),False),
   StructField ("OriginAirportID", IntegerType(),False),
    StructField("DepDelay", IntegerType(),False),
    StructField("ArrDelay", IntegerType(),False)
]
)

In [14]:
df=spark.read.csv("C:/Users/User/Desktop/Data/flights.csv", header=True, schema=flightschema)

In [15]:
df.show(5)

+----------+---------+-------+---------------+--------+--------+
|DayofMonth|DayofWeek|Carrier|OriginAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+--------+--------+
|        19|        5|     DL|          11433|   13303|      -3|
|        19|        5|     DL|          14869|   12478|       0|
|        19|        5|     DL|          14057|   14869|      -4|
|        19|        5|     DL|          15016|   11433|      28|
|        19|        5|     DL|          11193|   12892|      -6|
+----------+---------+-------+---------------+--------+--------+
only showing top 5 rows



In [16]:
df.printSchema()

root
 |-- DayofMonth: integer (nullable = true)
 |-- DayofWeek: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)



# select Some import data for Classification features and change arrival delay int0 binary class 
* late
* not late